In [52]:
import os
import numpy as np
import pandas as pd
import time
import platform
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from keras import regularizers
seed=2023

In [53]:
pd.set_option('display.max_columns',100)

In [54]:
# If words and punctuation marks are joined together, put a space between them.
def add_space_around_punctuation(texts):

    texts = [text.lower() for text in texts]

    punctuation_chars = r'!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'

    pattern_before = re.compile(r'(?<!\s)([' + re.escape(punctuation_chars) + r'])')
    pattern_after = re.compile(r'([' + re.escape(punctuation_chars) + r'])(?!\s|$)')

    texts_with_spaces = [pattern_before.sub(r' \1', text) for text in texts]
    texts_with_spaces = [pattern_after.sub(r'\1 ', text) for text in texts_with_spaces]

    texts_with_single_spaces = [re.sub(r'\s+', ' ', text) for text in texts_with_spaces]

    return texts_with_single_spaces

In [55]:
df = pd.read_csv('E:/CNSI_project/demo_and_data_5.0/Processed_data/MIMIC/aim1_2_merged1_baidu.csv')
df

,subject_id,hadm_id,chartdate,Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L",Blood Gas~Blood~Free Calcium~50808~mmol/L,Blood Gas~Blood~Glucose~50809~mg/dL,"Blood Gas~Blood~Hematocrit, Calculated~50810~%",Blood Gas~Blood~Hemoglobin~50811~g/dL,Blood Gas~Blood~Lactate~50813~mmol/L,Blood Gas~Blood~Methemoglobin~50814~%,Blood Gas~Blood~O2 Flow~50815~L/min,Blood Gas~Blood~Oxygen~50816~%,Blood Gas~Blood~Oxygen Saturation~50817~%,Blood Gas~Blood~pCO2~50818~mm Hg,Blood Gas~Blood~PEEP~50819~,Blood Gas~Blood~pH~50820~units,Blood Gas~Blood~pO2~50821~mm Hg,"Blood Gas~Blood~Potassium, Whole Blood~50822~mEq/L",Blood Gas~Blood~Required O2~50823~,"Blood Gas~Blood~Sodium, Whole Blood~50824~mEq/L",Blood Gas~Blood~Temperature~50825~,Blood Gas~Blood~Tidal Volume~50826~,Chemistry~Blood~% Hemoglobin A1c~50852~%,Chemistry~Blood~25-OH Vitamin D~50853~ng/mL,Chemistry~Blood~Acetaminophen~50856~ug/mL,Chemistry~Blood~Alanine Aminotransferase (ALT)~50861~IU/L,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,Chemistry~Blood~Alpha-Fetoprotein~50864~ng/mL,Chemistry~Blood~Amikacin~50865~ug/mL,Chemistry~Blood~Ammonia~50866~umol/L,Chemistry~Blood~Amylase~50867~IU/L,Chemistry~Blood~Anion Gap~50868~mEq/L,Chemistry~Blood~Anti-DGP (IgA/IgG)~50869~units,"Chemistry~Blood~Anti-Gliadin Antibody, IgA~50870~units",Chemistry~Blood~Anti-Thyroglobulin Antibodies~50877~IU/mL,Chemistry~Blood~Asparate Aminotransferase (AST)~50878~IU/L,Chemistry~Blood~Beta-2 Microglobulin~50881~mg/L,Chemistry~Blood~Bicarbonate~50882~mEq/L,"Chemistry~Blood~Bilirubin, Direct~50883~mg/dL","Chemistry~Blood~Bilirubin, Indirect~50884~mg/dL","Chemistry~Blood~Bilirubin, Total~50885~mg/dL",Chemistry~Blood~C-Reactive Protein~50889~mg/L_,Chemistry~Blood~C3~50890~mg/dL,Chemistry~Blood~C4~50891~mg/dL,...,Hematology~Blood~dRVVT - Confirmation~52107~,Hematology~Blood~dRVVT - Normalized Ratio~52108~,Hematology~Blood~dRVVT - Screen~52109~,Hematology~Blood~Immature Granulocytes~52135~%,Hematology~Blood~RDW-SD~52172~fL_,Hematology~Blood~SCT - Confirmation~52177~,Hematology~Blood~SCT - Normalized Ratio~52178~,Hematology~Blood~SCT - Screen~52179~,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%,Hematology~Blood~Absolute Lymphocyte Count~52769~#/uL,gender,age,culture,long_title,icd_title,chiefcomplaint,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,Allergies,Chief Complaint,History of Present Illness,Past Medical History,Family History,hospital_expire_flag
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [56]:
# Combine two ICDs
df0 = df.copy()
temp1 = df0['long_title'].values
temp2 = df0['icd_title'].values
temp3 = []
for i in range(df0.shape[0]):
    if pd.isna(temp1[i]) & pd.isna(temp2[i]):
        temp3.append(np.nan)
    elif pd.isna(temp1[i]):
        temp3.append(temp2[i])
    elif pd.isna(temp2[i]):
        temp3.append(temp1[i])
    elif temp1[i].lower() in temp2[i].lower():
        temp3.append(temp2[i])
    elif temp2[i].lower() in temp1[i].lower():
        temp3.append(temp1[i])
    else:
        temp3.append(temp1[i]+', '+temp2[i])
df0['icd_title'] = temp3
df0

,subject_id,hadm_id,chartdate,Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L",Blood Gas~Blood~Free Calcium~50808~mmol/L,Blood Gas~Blood~Glucose~50809~mg/dL,"Blood Gas~Blood~Hematocrit, Calculated~50810~%",Blood Gas~Blood~Hemoglobin~50811~g/dL,Blood Gas~Blood~Lactate~50813~mmol/L,Blood Gas~Blood~Methemoglobin~50814~%,Blood Gas~Blood~O2 Flow~50815~L/min,Blood Gas~Blood~Oxygen~50816~%,Blood Gas~Blood~Oxygen Saturation~50817~%,Blood Gas~Blood~pCO2~50818~mm Hg,Blood Gas~Blood~PEEP~50819~,Blood Gas~Blood~pH~50820~units,Blood Gas~Blood~pO2~50821~mm Hg,"Blood Gas~Blood~Potassium, Whole Blood~50822~mEq/L",Blood Gas~Blood~Required O2~50823~,"Blood Gas~Blood~Sodium, Whole Blood~50824~mEq/L",Blood Gas~Blood~Temperature~50825~,Blood Gas~Blood~Tidal Volume~50826~,Chemistry~Blood~% Hemoglobin A1c~50852~%,Chemistry~Blood~25-OH Vitamin D~50853~ng/mL,Chemistry~Blood~Acetaminophen~50856~ug/mL,Chemistry~Blood~Alanine Aminotransferase (ALT)~50861~IU/L,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,Chemistry~Blood~Alpha-Fetoprotein~50864~ng/mL,Chemistry~Blood~Amikacin~50865~ug/mL,Chemistry~Blood~Ammonia~50866~umol/L,Chemistry~Blood~Amylase~50867~IU/L,Chemistry~Blood~Anion Gap~50868~mEq/L,Chemistry~Blood~Anti-DGP (IgA/IgG)~50869~units,"Chemistry~Blood~Anti-Gliadin Antibody, IgA~50870~units",Chemistry~Blood~Anti-Thyroglobulin Antibodies~50877~IU/mL,Chemistry~Blood~Asparate Aminotransferase (AST)~50878~IU/L,Chemistry~Blood~Beta-2 Microglobulin~50881~mg/L,Chemistry~Blood~Bicarbonate~50882~mEq/L,"Chemistry~Blood~Bilirubin, Direct~50883~mg/dL","Chemistry~Blood~Bilirubin, Indirect~50884~mg/dL","Chemistry~Blood~Bilirubin, Total~50885~mg/dL",Chemistry~Blood~C-Reactive Protein~50889~mg/L_,Chemistry~Blood~C3~50890~mg/dL,Chemistry~Blood~C4~50891~mg/dL,...,Hematology~Blood~dRVVT - Confirmation~52107~,Hematology~Blood~dRVVT - Normalized Ratio~52108~,Hematology~Blood~dRVVT - Screen~52109~,Hematology~Blood~Immature Granulocytes~52135~%,Hematology~Blood~RDW-SD~52172~fL_,Hematology~Blood~SCT - Confirmation~52177~,Hematology~Blood~SCT - Normalized Ratio~52178~,Hematology~Blood~SCT - Screen~52179~,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%,Hematology~Blood~Absolute Lymphocyte Count~52769~#/uL,gender,age,culture,long_title,icd_title,chiefcomplaint,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,Allergies,Chief Complaint,History of Present Illness,Past Medical History,Family History,hospital_expire_flag
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [57]:
# Combined two main complaints
temp1 = df0['chiefcomplaint'].values
temp2 = df0['Chief Complaint'].values
temp3 = []
for i in range(df0.shape[0]):
    if pd.isna(temp1[i]) & pd.isna(temp2[i]):
        temp3.append(np.nan)
    elif pd.isna(temp1[i]):
        temp3.append(temp2[i])
    elif pd.isna(temp2[i]):
        temp3.append(temp1[i])
    elif temp1[i].lower() in temp2[i].lower():
        temp3.append(temp2[i])
    elif temp2[i].lower() in temp1[i].lower():
        temp3.append(temp1[i])
    else:
        temp3.append(temp1[i]+', '+temp2[i])
df0['ChiefComplaint_combined'] = temp3
df0 = df0.drop(['chiefcomplaint', 'Chief Complaint'], axis=1)
df0.rename(columns={'ChiefComplaint_combined':'Chief Complaint'},inplace=True)
df0

,subject_id,hadm_id,chartdate,Blood Gas~Blood~Alveolar-arterial Gradient~50801~mm Hg,Blood Gas~Blood~Base Excess~50802~mEq/L,"Blood Gas~Blood~Calculated Bicarbonate, Whole Blood~50803~mEq/L",Blood Gas~Blood~Calculated Total CO2~50804~mEq/L,Blood Gas~Blood~Carboxyhemoglobin~50805~%,"Blood Gas~Blood~Chloride, Whole Blood~50806~mEq/L",Blood Gas~Blood~Free Calcium~50808~mmol/L,Blood Gas~Blood~Glucose~50809~mg/dL,"Blood Gas~Blood~Hematocrit, Calculated~50810~%",Blood Gas~Blood~Hemoglobin~50811~g/dL,Blood Gas~Blood~Lactate~50813~mmol/L,Blood Gas~Blood~Methemoglobin~50814~%,Blood Gas~Blood~O2 Flow~50815~L/min,Blood Gas~Blood~Oxygen~50816~%,Blood Gas~Blood~Oxygen Saturation~50817~%,Blood Gas~Blood~pCO2~50818~mm Hg,Blood Gas~Blood~PEEP~50819~,Blood Gas~Blood~pH~50820~units,Blood Gas~Blood~pO2~50821~mm Hg,"Blood Gas~Blood~Potassium, Whole Blood~50822~mEq/L",Blood Gas~Blood~Required O2~50823~,"Blood Gas~Blood~Sodium, Whole Blood~50824~mEq/L",Blood Gas~Blood~Temperature~50825~,Blood Gas~Blood~Tidal Volume~50826~,Chemistry~Blood~% Hemoglobin A1c~50852~%,Chemistry~Blood~25-OH Vitamin D~50853~ng/mL,Chemistry~Blood~Acetaminophen~50856~ug/mL,Chemistry~Blood~Alanine Aminotransferase (ALT)~50861~IU/L,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,Chemistry~Blood~Alpha-Fetoprotein~50864~ng/mL,Chemistry~Blood~Amikacin~50865~ug/mL,Chemistry~Blood~Ammonia~50866~umol/L,Chemistry~Blood~Amylase~50867~IU/L,Chemistry~Blood~Anion Gap~50868~mEq/L,Chemistry~Blood~Anti-DGP (IgA/IgG)~50869~units,"Chemistry~Blood~Anti-Gliadin Antibody, IgA~50870~units",Chemistry~Blood~Anti-Thyroglobulin Antibodies~50877~IU/mL,Chemistry~Blood~Asparate Aminotransferase (AST)~50878~IU/L,Chemistry~Blood~Beta-2 Microglobulin~50881~mg/L,Chemistry~Blood~Bicarbonate~50882~mEq/L,"Chemistry~Blood~Bilirubin, Direct~50883~mg/dL","Chemistry~Blood~Bilirubin, Indirect~50884~mg/dL","Chemistry~Blood~Bilirubin, Total~50885~mg/dL",Chemistry~Blood~C-Reactive Protein~50889~mg/L_,Chemistry~Blood~C3~50890~mg/dL,Chemistry~Blood~C4~50891~mg/dL,...,Hematology~Blood~Absolute Neutrophil Count~52075~K/uL_,Hematology~Blood~dRVVT - Confirmation~52107~,Hematology~Blood~dRVVT - Normalized Ratio~52108~,Hematology~Blood~dRVVT - Screen~52109~,Hematology~Blood~Immature Granulocytes~52135~%,Hematology~Blood~RDW-SD~52172~fL_,Hematology~Blood~SCT - Confirmation~52177~,Hematology~Blood~SCT - Normalized Ratio~52178~,Hematology~Blood~SCT - Screen~52179~,Hematology~Cerebrospinal Fluid~Atypical Lymphocytes~52221~%,Hematology~Cerebrospinal Fluid~Bands~52223~%,Hematology~Cerebrospinal Fluid~Basophils~52225~%,Hematology~Cerebrospinal Fluid~Blasts~52227~%,Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,"Hematology~Cerebrospinal Fluid~Hematocrit, CSF~52257~%",Hematology~Cerebrospinal Fluid~Lining Cell~52263~%,Hematology~Cerebrospinal Fluid~Lymphs~52264~%,Hematology~Cerebrospinal Fluid~Macrophage~52266~%,Hematology~Cerebrospinal Fluid~Mesothelial cells~52268~%,Hematology~Cerebrospinal Fluid~Metamyelocytes~52270~%,Hematology~Cerebrospinal Fluid~Monocytes~52272~%,Hematology~Cerebrospinal Fluid~Myelocytes~52274~%,Hematology~Cerebrospinal Fluid~NRBC~52276~%,Hematology~Cerebrospinal Fluid~Other~52278~%,Hematology~Cerebrospinal Fluid~Plasma~52279~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,Hematology~Cerebrospinal Fluid~Promyelocytes~52283~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL","Hematology~Cerebrospinal Fluid~Total Nucleated Cells, CSF~52286~#/uL",Hematology~Cerebrospinal Fluid~Young~52288~%,Hematology~Blood~Absolute Lymphocyte Count~52769~#/uL,gender,age,culture,long_title,icd_title,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,acuity,Allergies,History of Present Illness,Past Medical History,Family History,hospital_expire_flag,Chief Complaint
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN

In [58]:
# Collate all possible columns of aim2
# Load the data selected by lasso
df_temp = pd.read_csv("E:/CNSI_project/demo_and_data_5.0/Processed_data/MIMIC/aim2_LASSOed.csv")

cols = ['subject_id','hadm_id','chartdate'] + df_temp.columns.tolist()[:-1] + ['Chief Complaint', 'icd_title', 'History of Present Illness', 'Past Medical History', 'Allergies', 'Family History', 'hospital_expire_flag']
df0 = df0[cols]
df0

,subject_id,hadm_id,chartdate,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,"Chemistry~Blood~Calcium, Total~50893~mg/dL",Chemistry~Blood~Chloride~50902~mEq/L,Chemistry~Blood~Lactate Dehydrogenase (LD)~50954~IU/L,Chemistry~Blood~Phosphate~50970~mg/dL,Chemistry~Blood~Urea Nitrogen~51006~mg/dL,Hematology~Blood~Eosinophils~51200~%,Hematology~Blood~INR(PT)~51237~,Hematology~Blood~MCV~51250~fL,Hematology~Blood~Platelet Count~51265~K/uL,Hematology~Blood~PT~51274~sec,Hematology~Blood~RDW~51277~%,Hematology~Blood~Red Blood Cells~51279~m/uL,Hematology~Blood~White Blood Cells~51301~K/uL,"Chemistry~Cerebrospinal Fluid~Lactate Dehydrogenase, CSF~51795~IU/L","Chemistry~Cerebrospinal Fluid~Total Protein, CSF~51802~mg/dL",Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL",age,resprate,acuity,Chief Complaint,icd_title,History of Present Illness,Past Medical History,Allergies,Family History,hospital_expire_flag
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,0.0,5.0,55,16.0,3.0,"LEFT HAND PAINS, Left hand and face numbness, ...",Cerebral edema; Compression of brain; Family h...,Mrs. ___ is a ___ y/o F from ___ with history ...,Multiple sclerosis,Penicillins,"Mother with pancreatic cancer, brother-lung ca...",0.0
1,10002557,NaN,2147-03-26,NaN,85.0,NaN,108.0,NaN,NaN,15.0,0.2,1.1,92.0,226.0,13.0,15.0,3.83,6.7,NaN,29.0,NaN,0.0,2.0,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10002559,NaN,2179-06-05,3.6,78.0,7.8,107.0,187.0,4.1,10.0,0.1,NaN,89.0,198.0,NaN,12.3,4.13,12.7,NaN,24.0,NaN,17.0,334.0,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10003019,21616816.0,2174-09-18,2.6,NaN,10.4,102.0,NaN,2.8,32.0,NaN,NaN,78.0,557.0,NaN,18.3,4.04,16.6,NaN,75.0,NaN,10.0,430.0,72,16.0,2.0,"FEVER/MS CHANGES, confusion, rehab placement","Acute kidney failure, unspecified; Asthma, uns...","___ male w/ PMH of sarcoidosis, sleep apnea, n...","1. Sarcoidosis, dx skin bx:intestinal & pulmon...",Oxycodone / Ragweed,"Mother:___, cardiac disease. Father:diverticul...",0.0
4,10005808,26543907.0,2119-01-02,NaN,48.0,9.3,105.0,NaN,2.5,18.0,3.2,NaN,87.0,256.0,NaN,13.9,5.03,9.8,NaN,72.0,NaN,0.0,510.0,53,NaN,NaN,limb weakness and tingling,Acute infective polyneuritis,Mr. ___ is a ___ year old right handed gentlem...,Hiatal hernia in ___ Occasional heartburn,No Known Allergies / Adverse Drug Reactions,He has no family history of autoimmune disease...,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11854,19994772,NaN,2181-03-13,NaN,85.0,NaN,106.0,180.0,NaN,18.0,0.6,NaN,93.0,218.0,NaN,12.8,3.85,14.4,57.0,590.0,NaN,90.0,250.0,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11855,19995080,23057109.0,2147-07-13,2.3,961.0,7.5,105.0,NaN,2.3,17.0,0.3,1.1,94.0,377.0,12.7,15.7,3.52,3.8,NaN,44.0,NaN,8.0,1.0,59,NaN,NaN,Confusion- Altered Mental Status- transfer fro...,Calculus of gallbladder and bile duct without ...,___ M with AIDS and med non-compliance (CD4 20...,"- AIDS off ART prescribed for Truvada T qd, Ka...",Penicillins,Father died of AD at age ___. Mother died of D...,0.0
11856,19997843,NaN,2120-11-18,3.4,171.0,7.5,113.0,453.0,3.0,7.0,0.1,1.4,104.0,88.0,14.9,14.3,3.29,5.6,NaN,21.0,NaN,9.0,4.0,47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11857,19999784,24935234.0,2119-07-07,4.3,NaN,NaN,NaN,102.0,NaN,NaN,0.6,NaN,84.0,275.0,NaN,14.6,4.65,5.0,23.0,146.0,NaN,1.0,4.0,57,16.0,2.0,"Abnormal MRI, weakness",Abnormal cytological findings in cerebrospinal...,Mr. ___ is a ___ without significant medical h...,PAST ONCOLOGIC HISTORY:___ is a ___ right-hand...,No Known Allergies / Adverse Drug Reactions,Father had prostate cancer. Denies otherwise ...,0.0


In [59]:
df1 = df0.copy()
df1 = df1.dropna(subset=['hospital_expire_flag'], axis=0)
df1

,subject_id,hadm_id,chartdate,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,"Chemistry~Blood~Calcium, Total~50893~mg/dL",Chemistry~Blood~Chloride~50902~mEq/L,Chemistry~Blood~Lactate Dehydrogenase (LD)~50954~IU/L,Chemistry~Blood~Phosphate~50970~mg/dL,Chemistry~Blood~Urea Nitrogen~51006~mg/dL,Hematology~Blood~Eosinophils~51200~%,Hematology~Blood~INR(PT)~51237~,Hematology~Blood~MCV~51250~fL,Hematology~Blood~Platelet Count~51265~K/uL,Hematology~Blood~PT~51274~sec,Hematology~Blood~RDW~51277~%,Hematology~Blood~Red Blood Cells~51279~m/uL,Hematology~Blood~White Blood Cells~51301~K/uL,"Chemistry~Cerebrospinal Fluid~Lactate Dehydrogenase, CSF~51795~IU/L","Chemistry~Cerebrospinal Fluid~Total Protein, CSF~51802~mg/dL",Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL",age,resprate,acuity,Chief Complaint,icd_title,History of Present Illness,Past Medical History,Allergies,Family History,hospital_expire_flag
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,0.0,5.0,55,16.0,3.0,"LEFT HAND PAINS, Left hand and face numbness, ...",Cerebral edema; Compression of brain; Family h...,Mrs. ___ is a ___ y/o F from ___ with history ...,Multiple sclerosis,Penicillins,"Mother with pancreatic cancer, brother-lung ca...",0.0
3,10003019,21616816.0,2174-09-18,2.6,NaN,10.4,102.0,NaN,2.8,32.0,NaN,NaN,78.0,557.0,NaN,18.3,4.04,16.6,NaN,75.0,NaN,10.0,430.0,72,16.0,2.0,"FEVER/MS CHANGES, confusion, rehab placement","Acute kidney failure, unspecified; Asthma, uns...","___ male w/ PMH of sarcoidosis, sleep apnea, n...","1. Sarcoidosis, dx skin bx:intestinal & pulmon...",Oxycodone / Ragweed,"Mother:___, cardiac disease. Father:diverticul...",0.0
4,10005808,26543907.0,2119-01-02,NaN,48.0,9.3,105.0,NaN,2.5,18.0,3.2,NaN,87.0,256.0,NaN,13.9,5.03,9.8,NaN,72.0,NaN,0.0,510.0,53,NaN,NaN,limb weakness and tingling,Acute infective polyneuritis,Mr. ___ is a ___ year old right handed gentlem...,Hiatal hernia in ___ Occasional heartburn,No Known Allergies / Adverse Drug Reactions,He has no family history of autoimmune disease...,0.0
5,10006269,27357430.0,2124-06-26,4.9,80.0,NaN,95.0,NaN,NaN,15.0,0.0,NaN,66.0,225.0,NaN,20.1,4.66,10.0,NaN,100.0,NaN,1.0,2.0,51,19.0,3.0,"Fever, Headache",Duodenitis without bleeding; Essential (primar...,NaN,NaN,No Known Allergies / Adverse Drug Reactions,NaN,0.0
9,10014078,25809882.0,2166-08-22,NaN,NaN,8.1,117.0,NaN,2.8,7.0,NaN,NaN,76.0,201.0,NaN,13.7,4.32,6.9,NaN,75.0,NaN,27.0,1.0,60,NaN,NaN,seizure,Benign neoplasm of cerebral meninges; Diabetes...,The pt is a ___ year-old woman who presents as...,hx of atypical meningioma - grade 2 - resected...,No Known Allergies / Adverse Drug Reactions,noncontributory,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11852,19994772,29219051.0,2181-04-18,NaN,NaN,NaN,103.0,150.0,NaN,9.0,0.3,NaN,92.0,325.0,NaN,12.7,4.02,10.7,NaN,244.0,NaN,88.0,0.0,55,20.0,2.0,Aletered mental status and fever,Abnormal involuntary movements; Candidiasis of...,___ is a ___ woman with,PAST MEDICAL HISTORY:Recurrent glioblastoma in...,Keflex,Non-contributory.,0.0
11853,19994772,29606061.0,2180-12-18,NaN,NaN,9.7,103.0,NaN,3.3,16.0,0.2,NaN,93.0,257.0,NaN,12.4,3.46,7.2,NaN,295.0,NaN,100.0,5100.0,54,NaN,NaN,Recurrent glioblastoma.,"Cerebral aneurysm, nonruptured; Compression of...",Ms. ___ is a ___ y/o F s/p resection of gliobl...,Hyperlipidemia,___,non-contributory,0.0
11855,19995080,23057109.0,2147-07-13,2.3,961.0,7.5,105.0,NaN,2.3,17.0,0.3,1.1,94.0,377.0,12.7,15.7,3.52,3.8,NaN,44.0,NaN,8.0,1.0,59,NaN,NaN,Confusion- Altered Mental Status- transfer fro...,Calculus of gallbladder and bile duct without ...,___ M with AIDS and med non-compliance (CD4 20...,"- AIDS off ART prescribed for Truvada T qd, Ka...",Penicillins,Father died of AD at age ___. Mother died of D.

In [60]:
# Same subject_id, but different endings
groups2 = df1.groupby(['subject_id'])['hospital_expire_flag'].apply(lambda x: x.dropna().nunique()>1)
filtered_data2 = groups2[groups2].reset_index()[['subject_id']]
print(f"相同的subject_id，而出现不同结果的病人有{filtered_data2.shape[0]}个。")
MultiResultsSamples = df1.merge(filtered_data2, on=['subject_id'], how='inner')
MultiResultsSamples = MultiResultsSamples.sort_values(by=['subject_id','chartdate'], ascending=True)
MultiResultsSamples["chartdate0"] = pd.to_datetime(MultiResultsSamples["chartdate"], format='%Y/%m/%d')
MultiResultsSamples

相同的subject_id，而出现不同结果的病人有32个。


,subject_id,hadm_id,chartdate,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,"Chemistry~Blood~Calcium, Total~50893~mg/dL",Chemistry~Blood~Chloride~50902~mEq/L,Chemistry~Blood~Lactate Dehydrogenase (LD)~50954~IU/L,Chemistry~Blood~Phosphate~50970~mg/dL,Chemistry~Blood~Urea Nitrogen~51006~mg/dL,Hematology~Blood~Eosinophils~51200~%,Hematology~Blood~INR(PT)~51237~,Hematology~Blood~MCV~51250~fL,Hematology~Blood~Platelet Count~51265~K/uL,Hematology~Blood~PT~51274~sec,Hematology~Blood~RDW~51277~%,Hematology~Blood~Red Blood Cells~51279~m/uL,Hematology~Blood~White Blood Cells~51301~K/uL,"Chemistry~Cerebrospinal Fluid~Lactate Dehydrogenase, CSF~51795~IU/L","Chemistry~Cerebrospinal Fluid~Total Protein, CSF~51802~mg/dL",Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL",age,resprate,acuity,Chief Complaint,icd_title,History of Present Illness,Past Medical History,Allergies,Family History,hospital_expire_flag,chartdate0
0,10073847,20508747.0,2134-04-06,NaN,107.0,9.0,105.0,379.0,3.9,10.0,0.0,NaN,90.0,193.0,NaN,19.8,4.01,9.3,25.0,49.0,NaN,0.0,1.0,53,NaN,NaN,Admission for EPOCH therapy for DLBCL.,Acute venous embolism and thrombosis of subcla...,Mr. ___ is a ___ year old gentleman with histo...,HEMATOLOGIC HISTORY:-___ - Presented to OSH wi...,Erythromycin Base,Father with colon cancer. No history of hemat...,0.0,2134-04-06
3,10073847,28332330.0,2134-04-30,4.3,81.0,8.7,106.0,230.0,3.7,15.0,NaN,NaN,93.0,331.0,NaN,20.8,3.84,13.2,23.0,56.0,NaN,40.0,245.0,53,NaN,NaN,Cycle #4 EPOCH for DLBCL.,Coronary atherosclerosis of native coronary ar...,Mr. ___ is a ___ yo man with a CAD and diastol...,HEMATOLOGIC HISTORY:-___ Presented to OSH with...,Erythromycin Base,Father with colon cancer. No history of hemat...,0.0,2134-04-30
2,10073847,27496246.0,2135-01-17,NaN,NaN,NaN,102.0,NaN,NaN,19.0,0.2,1.2,90.0,228.0,12.5,14.6,4.96,14.5,NaN,1424.0,NaN,3.0,100.0,54,20.0,2.0,"LE WEAKNESS, leg pain",Coronary atherosclerosis of native coronary ar...,OMED,HEMATOLOGIC HISTORY:-___ Presented to OSH with...,Erythromycin Base,Father with hx of colon cancer. No family hx o...,0.0,2135-01-17
4,10073847,29425640.0,2135-06-24,NaN,59.0,7.8,104.0,NaN,4.2,15.0,0.0,1.2,107.0,111.0,12.5,23.7,2.49,6.5,31.0,185.0,NaN,0.0,0.0,54,NaN,NaN,MTX infusion,Coronary atherosclerosis of native coronary ar...,"___ yo M with aggressive, double hit DLBCL. He...",TREATMENT HISTORY:Cycle 1 -R-EPOCH -___ (first...,Erythromycin Base,Father with hx of colon cancer. No family hx o...,0.0,2135-06-24
1,10073847,22194617.0,2135-12-28,NaN,43.0,7.4,107.0,NaN,2.7,25.0,0.0,1.2,104.0,46.0,12.8,21.5,2.73,3.2,24.0,52.0,NaN,3.0,1.0,54,NaN,NaN,"Altered mental statusm, Hypotension",Acute graft-versus-host disease; Acute kidney ...,___ with double hit lymphoma s/p six cycles of...,TREATMENT HISTORY:Cycle 1 -R-EPOCH -___ (first...,Erythromycin Base / anchovies,Father with hx of colon cancer. No family hx o...,1.0,2135-12-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,19667522,22530452.0,2188-12-24,2.2,390.0,8.5,105.0,687.0,3.8,79.0,0.0,1.0,99.0,35.0,11.7,25.4,2.65,6.2,NaN,24.0,NaN,0.0,31.0,63,NaN,NaN,"fever, abdominal pain","Acute lymphoid leukemia, without mention of ha...",This is a ___ yo M with biphenotypic AML/ALL d...,# Biphenotypic AML/ALL diagnosed ___ --S/P fou...,Lipitor,"+CAD, father with MI @ ___. Uncle and father w...",1.0,2188-12-24
139,19928034,29606324.0,2148-10-09,NaN,NaN,NaN,99.0,NaN,NaN,10.0,NaN,NaN,107.0,455.0,NaN,18.6,2.76,10.4,NaN,79.0,NaN,0.0,2.0,57,18.0,2.0,"Visual changes, Double Vision","Diffuse large B-cell lymphoma, extranodal and ...",Mrs. ___ is ___ year-old lady with a history o...,PAST ONCOLOGIC HISTORY:-___:Referred to Dr ___...,Penicillins,Paternal aunt with breast CA. Sister with brea...,0.0,2148-10-09
137,19928034,22496641.0,2148-11-16,3.9,99.0,8.9,103.0

In [61]:
# (1) Manually count MultiResultsSamples for each patient Interval between current admission and last admission for in-hospital deaths
i = 31 # i refers to the first patient of the multiple outcome patients counted above, for a total of 32 patients
temp = MultiResultsSamples[MultiResultsSamples['subject_id']==filtered_data2.iloc[i,0]]
temp

,subject_id,hadm_id,chartdate,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,"Chemistry~Blood~Calcium, Total~50893~mg/dL",Chemistry~Blood~Chloride~50902~mEq/L,Chemistry~Blood~Lactate Dehydrogenase (LD)~50954~IU/L,Chemistry~Blood~Phosphate~50970~mg/dL,Chemistry~Blood~Urea Nitrogen~51006~mg/dL,Hematology~Blood~Eosinophils~51200~%,Hematology~Blood~INR(PT)~51237~,Hematology~Blood~MCV~51250~fL,Hematology~Blood~Platelet Count~51265~K/uL,Hematology~Blood~PT~51274~sec,Hematology~Blood~RDW~51277~%,Hematology~Blood~Red Blood Cells~51279~m/uL,Hematology~Blood~White Blood Cells~51301~K/uL,"Chemistry~Cerebrospinal Fluid~Lactate Dehydrogenase, CSF~51795~IU/L","Chemistry~Cerebrospinal Fluid~Total Protein, CSF~51802~mg/dL",Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL",age,resprate,acuity,Chief Complaint,icd_title,History of Present Illness,Past Medical History,Allergies,Family History,hospital_expire_flag,chartdate0
139,19928034,29606324.0,2148-10-09,NaN,NaN,NaN,99.0,NaN,NaN,10.0,NaN,NaN,107.0,455.0,NaN,18.6,2.76,10.4,NaN,79.0,NaN,0.0,2.0,57,18.0,2.0,"Visual changes, Double Vision","Diffuse large B-cell lymphoma, extranodal and ...",Mrs. ___ is ___ year-old lady with a history o...,PAST ONCOLOGIC HISTORY:-___:Referred to Dr ___...,Penicillins,Paternal aunt with breast CA. Sister with brea...,0.0,2148-10-09
137,19928034,22496641.0,2148-11-16,3.9,99.0,8.9,103.0,274.0,3.9,23.0,0.2,0.9,103.0,358.0,10.0,16.4,3.02,8.0,34.0,69.0,NaN,48.0,116.0,57,NaN,NaN,NaN,"Diffuse large B-cell lymphoma, extranodal and ...",Ms. ___ is ___ female with a history of ulcera...,PAST ONCOLOGIC HISTORY:-___:Referred to Dr ___...,Penicillins,Paternal aunt with breast CA. Sister with brea...,0.0,2148-11-16
138,19928034,27666802.0,2149-01-21,3.6,300.0,8.8,105.0,478.0,3.6,8.0,0.0,0.9,98.0,430.0,9.9,18.6,3.11,6.0,36.0,119.0,NaN,8.0,19.0,58,18.0,2.0,"Altered mental status, Dyslexia",Adverse effect of antineoplastic and immunosup...,Ms. ___ is a ___ year-old woman with history o...,"PAST ONCOLOGIC HISTORY:-___ Diagnosed -___, Cy...",Penicillins,A paternal aunt with breast cancer. Sister wit...,0.0,2149-01-21
140,19928034,29699477.0,2149-03-27,2.8,254.0,7.3,94.0,655.0,3.1,17.0,0.0,NaN,95.0,280.0,NaN,25.2,2.57,9.0,42.0,78.0,NaN,0.0,4.0,58,NaN,NaN,LFT abnormality,Adverse effect of antineoplastic and immunosup...,Ms. ___ is a ___ woman with a history of high-...,"PAST ONCOLOGIC HISTORY:--___, Cycle 1 Rituxan/...",Penicillins,A paternal aunt with breast cancer. Sister wit...,1.0,2149-03-27


In [62]:
# (2) Manually count MultiResultsSamples for each patient Interval between current admission and last admission for in-hospital deaths
temp['chartdate0'].loc[140] - temp['chartdate0'].loc[138]

Timedelta('65 days 00:00:00')

In [63]:
# (3) Manually count MultiResultsSamples for each patient Interval between current admission and last admission for in-hospital deaths
dic_subject = {10073847:187, 10533034:224, 10711182:2768, 11424506:191, 11644977:190, 12570381:54, 12674657:23, 13281088:27, 13624277:81, 13702880:227,
 13792123:113, 14948329:23, 14976423:85, 15696349:351, 16108683:156, 16133052:66, 16668491:164, 17190208:1720, 17267101:14, 17342098:71,
 17394776:31, 18137852:420, 18507022:361, 18569328:1481, 18955798:259, 19016010:51, 19232236:72, 19243413:22, 19493050:183, 19586042:95,
 19667522:35, 19928034:65
}

In [64]:
# The longest and shortest intervals are 2768 and 14 respectively.
from collections import OrderedDict
sorted_keys = sorted(dic_subject, key=dic_subject.get)
sorted_dict = OrderedDict((k, dic_subject[k]) for k in sorted_keys)
sorted_dict

OrderedDict([(17267101, 14),
             (19243413, 22),
             (12674657, 23),
             (14948329, 23),
             (13281088, 27),
             (17394776, 31),
             (19667522, 35),
             (19016010, 51),
             (12570381, 54),
             (19928034, 65),
             (16133052, 66),
             (17342098, 71),
             (19232236, 72),
             (13624277, 81),
             (14976423, 85),
             (19586042, 95),
             (13792123, 113),
             (16108683, 156),
             (16668491, 164),
             (19493050, 183),
             (10073847, 187),
             (11644977, 190),
             (11424506, 191),
             (10533034, 224),
             (13702880, 227),
             (18955798, 259),
             (15696349, 351),
             (18507022, 361),
             (18137852, 420),
             (18569328, 1481),
             (17190208, 1720),
             (10711182, 2768)])

In [65]:
# 0 cases of same hadm_id with different endings. Therefore, if the sample of aim2 is selected using hadm_id as the only indicator, there will be no conflict, whereas using subject_id as the selection indicator will be a conflict.
groups2 = df1.groupby(['hadm_id'])['hospital_expire_flag'].apply(lambda x: x.dropna().nunique()>1)
filtered_data2 = groups2[groups2].reset_index()[['hadm_id']]
print(f"相同的hadm_id，而出现不同结果的情况有{filtered_data2.shape[0]}个。")

相同的hadm_id，而出现不同结果的情况有0个。


In [66]:
def count_non_null_features(row):
    return row.count()

df2 = df1.groupby(['subject_id','hadm_id']).apply(lambda x: x.loc[x.apply(count_non_null_features, axis=1).idxmax()])
df2 = df2.reset_index(drop=True)
df2 = df2.sort_values(by=['subject_id','hadm_id','chartdate'], ascending=True)
df2

,subject_id,hadm_id,chartdate,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,"Chemistry~Blood~Calcium, Total~50893~mg/dL",Chemistry~Blood~Chloride~50902~mEq/L,Chemistry~Blood~Lactate Dehydrogenase (LD)~50954~IU/L,Chemistry~Blood~Phosphate~50970~mg/dL,Chemistry~Blood~Urea Nitrogen~51006~mg/dL,Hematology~Blood~Eosinophils~51200~%,Hematology~Blood~INR(PT)~51237~,Hematology~Blood~MCV~51250~fL,Hematology~Blood~Platelet Count~51265~K/uL,Hematology~Blood~PT~51274~sec,Hematology~Blood~RDW~51277~%,Hematology~Blood~Red Blood Cells~51279~m/uL,Hematology~Blood~White Blood Cells~51301~K/uL,"Chemistry~Cerebrospinal Fluid~Lactate Dehydrogenase, CSF~51795~IU/L","Chemistry~Cerebrospinal Fluid~Total Protein, CSF~51802~mg/dL",Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL",age,resprate,acuity,Chief Complaint,icd_title,History of Present Illness,Past Medical History,Allergies,Family History,hospital_expire_flag
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,0.0,5.0,55,16.0,3.0,"LEFT HAND PAINS, Left hand and face numbness, ...",Cerebral edema; Compression of brain; Family h...,Mrs. ___ is a ___ y/o F from ___ with history ...,Multiple sclerosis,Penicillins,"Mother with pancreatic cancer, brother-lung ca...",0.0
1,10003019,21616816.0,2174-09-18,2.6,NaN,10.4,102.0,NaN,2.8,32.0,NaN,NaN,78.0,557.0,NaN,18.3,4.04,16.6,NaN,75.0,NaN,10.0,430.0,72,16.0,2.0,"FEVER/MS CHANGES, confusion, rehab placement","Acute kidney failure, unspecified; Asthma, uns...","___ male w/ PMH of sarcoidosis, sleep apnea, n...","1. Sarcoidosis, dx skin bx:intestinal & pulmon...",Oxycodone / Ragweed,"Mother:___, cardiac disease. Father:diverticul...",0.0
2,10005808,26543907.0,2119-01-02,NaN,48.0,9.3,105.0,NaN,2.5,18.0,3.2,NaN,87.0,256.0,NaN,13.9,5.03,9.8,NaN,72.0,NaN,0.0,510.0,53,NaN,NaN,limb weakness and tingling,Acute infective polyneuritis,Mr. ___ is a ___ year old right handed gentlem...,Hiatal hernia in ___ Occasional heartburn,No Known Allergies / Adverse Drug Reactions,He has no family history of autoimmune disease...,0.0
3,10006269,27357430.0,2124-06-26,4.9,80.0,NaN,95.0,NaN,NaN,15.0,0.0,NaN,66.0,225.0,NaN,20.1,4.66,10.0,NaN,100.0,NaN,1.0,2.0,51,19.0,3.0,"Fever, Headache",Duodenitis without bleeding; Essential (primar...,NaN,NaN,No Known Allergies / Adverse Drug Reactions,NaN,0.0
4,10014078,25809882.0,2166-08-22,NaN,NaN,8.1,117.0,NaN,2.8,7.0,NaN,NaN,76.0,201.0,NaN,13.7,4.32,6.9,NaN,75.0,NaN,27.0,1.0,60,NaN,NaN,seizure,Benign neoplasm of cerebral meninges; Diabetes...,The pt is a ___ year-old woman who presents as...,hx of atypical meningioma - grade 2 - resected...,No Known Allergies / Adverse Drug Reactions,noncontributory,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5790,19994772,29219051.0,2181-04-18,NaN,NaN,NaN,103.0,150.0,NaN,9.0,0.3,NaN,92.0,325.0,NaN,12.7,4.02,10.7,NaN,244.0,NaN,88.0,0.0,55,20.0,2.0,Aletered mental status and fever,Abnormal involuntary movements; Candidiasis of...,___ is a ___ woman with,PAST MEDICAL HISTORY:Recurrent glioblastoma in...,Keflex,Non-contributory.,0.0
5791,19994772,29606061.0,2180-12-18,NaN,NaN,9.7,103.0,NaN,3.3,16.0,0.2,NaN,93.0,257.0,NaN,12.4,3.46,7.2,NaN,295.0,NaN,100.0,5100.0,54,NaN,NaN,Recurrent glioblastoma.,"Cerebral aneurysm, nonruptured; Compression of...",Ms. ___ is a ___ y/o F s/p resection of gliobl...,Hyperlipidemia,___,non-contributory,0.0
5792,19995080,23057109.0,2147-07-13,2.3,961.0,7.5,105.0,NaN,2.3,17.0,0.3,1.1,94.0,377.0,12.7,15.7,3.52,3.8,NaN,44.0,NaN,8.0,1.0,59,NaN,NaN,Confusion- Altered Mental Status- transfer fro...,Calculus of gallbladder and bile duct without ...,___ M with AIDS and med non-compliance (CD4 20...,"- AIDS off ART prescribed for Truvada T qd, Ka...",Penicillins,Father died of AD at age ___. Mother died of D...,

In [67]:
# Take only the last sample
df3 = df1.sort_values(by=['subject_id','hadm_id','chartdate'], ascending=True)
df3 = df3.drop_duplicates(subset=['subject_id','hadm_id'], keep='last')
df3

,subject_id,hadm_id,chartdate,Chemistry~Blood~Albumin~50862~g/dL,Chemistry~Blood~Alkaline Phosphatase~50863~IU/L,"Chemistry~Blood~Calcium, Total~50893~mg/dL",Chemistry~Blood~Chloride~50902~mEq/L,Chemistry~Blood~Lactate Dehydrogenase (LD)~50954~IU/L,Chemistry~Blood~Phosphate~50970~mg/dL,Chemistry~Blood~Urea Nitrogen~51006~mg/dL,Hematology~Blood~Eosinophils~51200~%,Hematology~Blood~INR(PT)~51237~,Hematology~Blood~MCV~51250~fL,Hematology~Blood~Platelet Count~51265~K/uL,Hematology~Blood~PT~51274~sec,Hematology~Blood~RDW~51277~%,Hematology~Blood~Red Blood Cells~51279~m/uL,Hematology~Blood~White Blood Cells~51301~K/uL,"Chemistry~Cerebrospinal Fluid~Lactate Dehydrogenase, CSF~51795~IU/L","Chemistry~Cerebrospinal Fluid~Total Protein, CSF~51802~mg/dL",Hematology~Cerebrospinal Fluid~Eosinophils~52256~%,Hematology~Cerebrospinal Fluid~Polys~52281~%,"Hematology~Cerebrospinal Fluid~RBC, CSF~52285~#/uL",age,resprate,acuity,Chief Complaint,icd_title,History of Present Illness,Past Medical History,Allergies,Family History,hospital_expire_flag
0,10001217,24597018.0,2157-11-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.0,NaN,0.0,5.0,55,16.0,3.0,"LEFT HAND PAINS, Left hand and face numbness, ...",Cerebral edema; Compression of brain; Family h...,Mrs. ___ is a ___ y/o F from ___ with history ...,Multiple sclerosis,Penicillins,"Mother with pancreatic cancer, brother-lung ca...",0.0
3,10003019,21616816.0,2174-09-18,2.6,NaN,10.4,102.0,NaN,2.8,32.0,NaN,NaN,78.0,557.0,NaN,18.3,4.04,16.6,NaN,75.0,NaN,10.0,430.0,72,16.0,2.0,"FEVER/MS CHANGES, confusion, rehab placement","Acute kidney failure, unspecified; Asthma, uns...","___ male w/ PMH of sarcoidosis, sleep apnea, n...","1. Sarcoidosis, dx skin bx:intestinal & pulmon...",Oxycodone / Ragweed,"Mother:___, cardiac disease. Father:diverticul...",0.0
4,10005808,26543907.0,2119-01-02,NaN,48.0,9.3,105.0,NaN,2.5,18.0,3.2,NaN,87.0,256.0,NaN,13.9,5.03,9.8,NaN,72.0,NaN,0.0,510.0,53,NaN,NaN,limb weakness and tingling,Acute infective polyneuritis,Mr. ___ is a ___ year old right handed gentlem...,Hiatal hernia in ___ Occasional heartburn,No Known Allergies / Adverse Drug Reactions,He has no family history of autoimmune disease...,0.0
5,10006269,27357430.0,2124-06-26,4.9,80.0,NaN,95.0,NaN,NaN,15.0,0.0,NaN,66.0,225.0,NaN,20.1,4.66,10.0,NaN,100.0,NaN,1.0,2.0,51,19.0,3.0,"Fever, Headache",Duodenitis without bleeding; Essential (primar...,NaN,NaN,No Known Allergies / Adverse Drug Reactions,NaN,0.0
9,10014078,25809882.0,2166-08-22,NaN,NaN,8.1,117.0,NaN,2.8,7.0,NaN,NaN,76.0,201.0,NaN,13.7,4.32,6.9,NaN,75.0,NaN,27.0,1.0,60,NaN,NaN,seizure,Benign neoplasm of cerebral meninges; Diabetes...,The pt is a ___ year-old woman who presents as...,hx of atypical meningioma - grade 2 - resected...,No Known Allergies / Adverse Drug Reactions,noncontributory,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11852,19994772,29219051.0,2181-04-18,NaN,NaN,NaN,103.0,150.0,NaN,9.0,0.3,NaN,92.0,325.0,NaN,12.7,4.02,10.7,NaN,244.0,NaN,88.0,0.0,55,20.0,2.0,Aletered mental status and fever,Abnormal involuntary movements; Candidiasis of...,___ is a ___ woman with,PAST MEDICAL HISTORY:Recurrent glioblastoma in...,Keflex,Non-contributory.,0.0
11853,19994772,29606061.0,2180-12-18,NaN,NaN,9.7,103.0,NaN,3.3,16.0,0.2,NaN,93.0,257.0,NaN,12.4,3.46,7.2,NaN,295.0,NaN,100.0,5100.0,54,NaN,NaN,Recurrent glioblastoma.,"Cerebral aneurysm, nonruptured; Compression of...",Ms. ___ is a ___ y/o F s/p resection of gliobl...,Hyperlipidemia,___,non-contributory,0.0
11855,19995080,23057109.0,2147-07-13,2.3,961.0,7.5,105.0,NaN,2.3,17.0,0.3,1.1,94.0,377.0,12.7,15.7,3.52,3.8,NaN,44.0,NaN,8.0,1.0,59,NaN,NaN,Confusion- Altered Mental Status- transfer fro...,Calculus of gallbladder and bile duct without ...,___ M with AIDS and med non-compliance (CD4 20...,"- AIDS off ART prescribed for Truvada T qd, Ka...",Penicillins,Father died of AD at age ___. Mother died of D.

In [68]:
df3['hospital_expire_flag'] = df3['hospital_expire_flag'].astype('int16')

In [69]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5795 entries, 0 to 11858
Data columns (total 33 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   subject_id                                                           5795 non-null   int64  
 1   hadm_id                                                              5795 non-null   float64
 2   chartdate                                                            5795 non-null   object 
 3   Chemistry~Blood~Albumin~50862~g/dL                                   1486 non-null   float64
 4   Chemistry~Blood~Alkaline Phosphatase~50863~IU/L                      2241 non-null   float64
 5   Chemistry~Blood~Calcium, Total~50893~mg/dL                           3966 non-null   float64
 6   Chemistry~Blood~Chloride~50902~mEq/L                                 4992 non-null   float64
 7   Chemi

In [70]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5795 entries, 0 to 5794
Data columns (total 33 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   subject_id                                                           5795 non-null   int64  
 1   hadm_id                                                              5795 non-null   float64
 2   chartdate                                                            5795 non-null   object 
 3   Chemistry~Blood~Albumin~50862~g/dL                                   1600 non-null   float64
 4   Chemistry~Blood~Alkaline Phosphatase~50863~IU/L                      2349 non-null   float64
 5   Chemistry~Blood~Calcium, Total~50893~mg/dL                           3991 non-null   float64
 6   Chemistry~Blood~Chloride~50902~mEq/L                                 5042 non-null   float64
 7   Chemis

In [71]:
df3.hospital_expire_flag.value_counts()

0    5500
1     295
Name: hospital_expire_flag, dtype: int64

In [72]:
df3.to_csv("E:/CNSI_project/demo_and_data_5.0/Processed_data/MIMIC/aim2_dataAll_filtered.csv", index=False)